# Load environment variables

In [63]:
from dotenv import load_dotenv

load_dotenv()

True

# Basic Tool in Langchain

## By using tool decorator

In [62]:
from langchain_core.tools import tool


@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers and returns the sum."""
    return a + b


add_numbers.run(tool_input={"a": 3, "b": 5})

8

In [ ]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor
from langchain.tools.render import render_text_description

# Define the tool for addition
@tool
def add_numbers(input: str) -> int:
    """Add two numbers given in 'num1 num2' format."""
    numbers = [int(s) for s in input.replace("+", " ").split() if s.isdigit()]
    result = sum(numbers)
    print(f"Tool called with input: {input}, result: {result}")
    return result

# Instantiate your LLM
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# Custom prompt explicitly instructing ReAct style
prompt_template = """
You are a helpful assistant.

Follow this approach strictly when answering the user's question:

- If you need to use a tool to answer the question, you must clearly specify an Action and Action Input.
- If you have enough information to answer directly, provide a Final Answer.

Here is an example of the REQUIRED format:

Question: What is 3 plus 5?
Thought: To find the sum, I must use the available tool add_numbers.
Action: add_numbers
Action Input: 3 5
Observation: 8
Final Answer: The sum of 3 and 5 is 8.

You have the following tools:
{tools}

Now, answer this question:

Question: {question}
{agent_scratchpad}
"""

# Create prompt template object
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["tools", "question", "agent_scratchpad"]
)

# Create the agent pipeline using ReActSingleInputOutputParser
llm_with_stop = llm.bind(stop=["\nObservation"])

agent_runnable = (
    {
        "tools": lambda x: render_text_description([add_numbers]),
        "question": lambda x: x["question"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

# Create AgentExecutor
agent_executor = AgentExecutor(agent=agent_runnable, tools=[add_numbers], verbose=True)

# Now we run the agent correctly with proper parsing
response = agent_executor.invoke({"question": "What is the sum of 3 and 5?"})
print("Final Response:", response["output"])



> Entering new AgentExecutor chain...
Thought: To find the sum, I must use the available tool add_numbers.
Action: add_numbers
Action Input: 3 5Tool called with input: 3 5, result: 8
8Final Answer: The sum of 3 and 5 is 8.

> Finished chain.
Final Response: The sum of 3 and 5 is 8.


## By using langchain agent

In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_openai import ChatOpenAI


def add_numbers(input: str) -> int:
    """Add two numbers and returns the sum."""
    numbers = [int(s) for s in input.replace("+", " ").split() if s.isdigit()]
    result = sum(numbers)
    print(f"Tool called with input: {input}, result: {result}")
    return result


add_tool = Tool(
    func=add_numbers,
    name="add_numbers",
    description="Function to add two numbers",
)

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

agent = initialize_agent(
    [add_tool],
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.invoke("add 2 and 3")



> Entering new AgentExecutor chain...
I need to add the numbers 2 and 3 together.

Action: add_numbers
Action Input: [2, 3][2, 3] <class 'str'>
a: 2, b: 3

Observation: 5
Thought:I now know the final answer
Final Answer: 5

> Finished chain.


{'input': 'add 2 and 3', 'output': '5'}

## Another example of using Langchain agent

In [ ]:
from langchain.agents import Tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType


# Define a custom tool
def my_tool(input_string: str) -> str:
    return f"Processed input: {input_string}"


# Wrap the tool
my_tool_instance = Tool(
    name="MyCustomTool",
    func=my_tool,
    description="This tool processes a string input",
)

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

tools = [my_tool_instance]
agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

result = agent.run("Give me some processed input for Langchain")
result



> Entering new AgentExecutor chain...
Thought: To produce a processed input for LangChain, I will need to apply the defined MyCustomTool to a sample string related to LangChain. Since the nature of the processing is not detailed, I'll select a generic simple string related to LangChain to transform using the tool.

Action: MyCustomTool
Action Input: "Explore advanced applications of LangChain technology."
Observation: Processed input: Explore advanced applications of LangChain technology.
Thought:I now know the final answer
Final Answer: The processed string for "LangChain" using MyCustomTool is "Explore advanced applications of LangChain technology."

> Finished chain.


'The processed string for "LangChain" using MyCustomTool is "Explore advanced applications of LangChain technology."'

# How to use multiple tools

In [ ]:
def tool_1(input_string: str) -> str:
    return f"Tool 1 processed: {input_string}"


def tool_2(input_string: str) -> str:
    return f"Tool 2 processed: {input_string}"


tool_1_instance = Tool(
    name="Tool1",
    func=tool_1,
    description="This tool processes input with Tool 1",
)

tool_2_instance = Tool(
    name="Tool2",
    func=tool_2,
    description="This tool processes input with Tool 2",
)

tools = [tool_1_instance, tool_2_instance]

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# Initialize agent with multiple tools
agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Run the chain with an input
result = agent.run("I want to use tool 2.")
print(result)



> Entering new AgentExecutor chain...
Since the user explicitly wants to use Tool 2, I will oblige.

Action: Tool2
Action Input: "I want to use tool 2."
Observation: Tool 2 processed: I want to use tool 2.
Thought:I now know the final answer
Final Answer: Tool 2 has been used as requested.

> Finished chain.
Tool 2 has been used as requested.


# Another example of using multiple tools, a bit complex with one tools output as the input of other tool

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain_openai import ChatOpenAI


# Define calculator function
def calculator(query: str) -> str:
    try:
        result = eval(query)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"


# Define factorial function
def factorial(num: str) -> int:
    num = int(num)
    if num < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    elif num == 0:
        return 1
    fact = 1
    for i in range(1, num + 1):
        fact *= i
    return fact


# Tools setup
calculator_tool = Tool(
    name="Calculator",
    func=calculator,
    description="Evaluates basic arithmetic expressions.",
)
factorial_tool = Tool(
    name="Factorial",
    func=factorial,
    description="Computes factorial of a given integer.",
)


# Initialize your OpenAI language model
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# Add all three tools to your agent
tools = [calculator_tool, factorial_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
)

# Run query
query = "What is 56 multiplied by 23 plus 12, divided by 100, and factorial of it?"
response = agent.run(query)
print(response)

/var/folders/k8/kzbt1g0x6jn_40b5dzhsl6wh0000gn/T/ipykernel_4184/3047223167.py:53: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)




> Entering new AgentExecutor chain...
First, I need to calculate the expression 56 multiplied by 23, add 12 to the result, and then divide by 100.

Action: Calculator
Action Input: (56 * 23 + 12) / 100
Observation: 13.0
Thought:The result of the arithmetic calculation is 13.0, which simplifies to 13. Next, I need to compute the factorial of 13.

Action: Factorial
Action Input: 13
Observation: 6227020800
Thought:I now know the final answer.

Final Answer: 6227020800

> Finished chain.
6227020800


# Using langchain tool with structure input to the tools, with multiple tools, where one tool used to get the langitude and longitude, and another tool to get the weather details based on langitude and longitude

In [ ]:
from dotenv import load_dotenv
from langchain.tools import StructuredTool
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
import requests
import os

load_dotenv()


# Define structured input schema for the tool.
class WeatherModel(BaseModel):
    latitude: float
    longitude: float


class LatitudeLongitudeModel(BaseModel):
    city_name: str


# Function matching the input schema
def get_weather(latitude: float, longitude: float) -> dict:
    """Fetch the weather details."""
    API_KEY = os.environ["WEATHER_API_KEY"]
    API_URL = f"https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={API_KEY}"
    return requests.get(API_URL).json()


def get_latitude_longitude(city_name: str) -> list:
    """Fetch the latitude and longitude."""
    API_KEY = os.environ["WEATHER_API_KEY"]
    API_URL = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name}&limit=3&appid={API_KEY}"
    return requests.get(API_URL).json()


# Define a structured tool using StructuredTool
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="get_weather",
    description="Fetch the weather details.",
    args_schema=WeatherModel,
)

latitude_longitude_tool = StructuredTool.from_function(
    func=get_latitude_longitude,
    name="get_latitude_longitude",
    description="Fetch the latitude and longitude.",
    args_schema=LatitudeLongitudeModel,
)

# Initialize the Chat model
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# Change agent type to OPENAI_FUNCTIONS to support multi-input structured tools.
agent = initialize_agent(
    tools=[latitude_longitude_tool, weather_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

# Invoke the agent
agent.invoke({"input": "What is the weather of Gwalior, India"})



> Entering new AgentExecutor chain...

Invoking: `get_latitude_longitude` with `{'city_name': 'Gwalior'}`


[{'name': 'Gwalior', 'local_names': {'mr': 'ग्वालियर', 'ar': 'قاليور', 'ur': 'گوالیار', 'ja': 'グワーリヤル', 'ml': 'ഗ്വാളിയർ', 'hi': 'ग्वालियर', 'kn': 'ಗ್ವಾಲಿಯರ್', 'uk': 'Ґваліор', 'be': 'Гваліёр', 'pa': 'ਗਵਾਲੀਅਰ', 'es': 'Gwalior', 'de': 'Gwalior', 'sr': 'Гвалиор', 'he': 'גוואליור', 'ru': 'Гвалиор', 'ta': 'குவாலியர்'}, 'lat': 26.2037247, 'lon': 78.1573628, 'country': 'IN', 'state': 'Madhya Pradesh'}]
Invoking: `get_weather` with `{'latitude': 26.2037247, 'longitude': 78.1573628}`


{'coord': {'lon': 78.1574, 'lat': 26.2037}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 295.91, 'feels_like': 294.71, 'temp_min': 295.91, 'temp_max': 295.91, 'pressure': 1012, 'humidity': 18, 'sea_level': 1012, 'grnd_level': 986}, 'visibility': 10000, 'wind': {'speed': 1.4, 'deg': 328, 'gust': 1.44}, 'clouds': {'all': 1}, 'dt':

{'input': 'What is the weather of Gwalior, India',
 'output': 'The current weather in Gwalior, India is clear with a temperature of 22.76°C (feels like 21.56°C). The humidity is at 18%, and there is a light wind blowing from the northwest at 1.4 m/s. The visibility is good, at 10,000 meters.'}